<a href="https://colab.research.google.com/github/SingularityNET-Archive/LLM-Development/blob/main/Vector_Store_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Store Creation

In [23]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [68]:
#!pip install jq

In [69]:
from langchain_community.document_loaders import JSONLoader

In [70]:
import json
from pathlib import Path
from pprint import pprint


file_path='/content/meeting-summaries-array.json'
data = json.loads(Path(file_path).read_text())

In [102]:
## pprint(data)

## Create vector (using FAISS)
FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other.

In [76]:
import json
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

# Load the JSON data
with open('/content/meeting-summaries-array.json', 'r') as file:
    data = json.load(file)

# Create a text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Split the data into chunks
chunks = []
for item in data:
    chunks.extend(text_splitter.split_text(str(item)))

# Convert the chunked data to Document objects
docs = [Document(page_content=chunk) for chunk in chunks]

# Iterate over the documents and print the content
for doc in docs:
    print(doc.page_content)
    print("-" * 20)

# Create the vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding=embeddings)

{'workgroup': 'Knowledge Base Workgroup', 'workgroup_id': '96f6ef88-5285-4ec6-8fe4-f2ab0347be00', 'meetingInfo': {'name': 'Biweekly', 'date': '2024-01-24', 'host': 'Tevo', 'documenter': 'Tevo', 'peoplePresent': 'Tevo, Shankar, Vanessa, Peter, André, love, Slate, Jade, Emotu, Sucren, Friday, Anthony, devon', 'purpose': 'Aggregating Ambassador Program assets and relevant information under GitBook', 'meetingVideoLink': 'https://miro.com/app/board/uXjVN-9yivE=/?moveToWidget=3458764574355478962&cot=10', 'miroBoardLink': 'https://miro.com/app/board/uXjVN-9yivE=/?moveToWidget=3458764574355478962&cot=10', 'otherMediaLink': 'https://app.dework.xyz/singularitynet-ambas/process-guild', 'workingDocs': [{'title': 'Knowledge Base WG', 'link': 'https://docs.google.com/document/d/1Lb9DwNSzmToyMhl5qZG9QEFipH9ZmlnDNaVU1jYeqFc/edit?usp=sharing'}]}, 'agendaItems': [{'agenda': 'Continues Onboarding', 'status': 'resolved', 'actionItems': [{'text': 'Create an onboarding feedback form that is mainly meant for

## vectorstore.similarity_search

In [79]:
# Ask a query
query = "Who is the host of the Writers Workgroup?"
results = vectorstore.similarity_search(query)

# Print the most relevant documents
print(f"Search results for: '{query}'")
for result in results:
    print(result.page_content)
    print("-" * 50)

Search results for: 'Who is the host of the Writers Workgroup?'
{'workgroup': 'Writers Workgroup', 'workgroup_id': '3506256d-ae6f-4147-a2fb-a4a76ebac109', 'meetingInfo': {'name': 'Weekly', 'date': '2024-04-12', 'host': 'Kenichi', 'documenter': 'devon', 'peoplePresent': 'Kenichi, Mikasa, cjfrankie , LadyTempestt, Slate, William, Gorga Siagian, ines ganina', 'purpose': 'The meeting focused on article updates, zealy updates , platforms for articles, and workgroup operations.', 'meetingVideoLink': 'https://us02web.zoom.us/j/89224946982?pwd=bVo0ckVyNlMxVmdxaUI1cUtUOWJDQT09', 'workingDocs': [{'title': 'Content editing test', 'link': 'https://docs.google.com/document/d/119vvftlF8IVEo-82kq3Fyq559WZwlgAeAej8SX43_uE/edit?usp=sharing'}, {'title': 'Customer-centric AI Solutions: article for Cogito by LadyTempestt', 'link': 'https://docs.google.com/document/d/1HjtlQRklpjbCB2qzVyENQ-mbZ9weZy862bBTlxxOnps/edit?usp=sharing'}, {'title': "Writers' WG budget proposal for Q1 2024", 'link': 'https://docs.g

## Some JSON Index Filtering

In [96]:
# Assuming 'data' is a list of dictionaries with each dictionary representing an item
for item in data:
    text_to_query = item['meetingInfo']  # Accessing the 'text' key
    # Now you can pass 'text_to_query' to your query function
    # For example, you can print it
    print("Querying:", text_to_query)
    # Or perform some other operation with 'text_to_query'


Querying: {'name': 'Biweekly', 'date': '2024-01-24', 'host': 'Tevo', 'documenter': 'Tevo', 'peoplePresent': 'Tevo, Shankar, Vanessa, Peter, André, love, Slate, Jade, Emotu, Sucren, Friday, Anthony, devon', 'purpose': 'Aggregating Ambassador Program assets and relevant information under GitBook', 'meetingVideoLink': 'https://miro.com/app/board/uXjVN-9yivE=/?moveToWidget=3458764574355478962&cot=10', 'miroBoardLink': 'https://miro.com/app/board/uXjVN-9yivE=/?moveToWidget=3458764574355478962&cot=10', 'otherMediaLink': 'https://app.dework.xyz/singularitynet-ambas/process-guild', 'workingDocs': [{'title': 'Knowledge Base WG', 'link': 'https://docs.google.com/document/d/1Lb9DwNSzmToyMhl5qZG9QEFipH9ZmlnDNaVU1jYeqFc/edit?usp=sharing'}]}
Querying: {'name': 'Biweekly', 'date': '2024-01-25', 'host': 'Tevo', 'documenter': 'Tevo', 'peoplePresent': 'Tevo, André, Miro, Emotu, Vanessa, Peter, Rojo, William, Headelf, Guilermo, Sucren', 'purpose': 'Structuring Ambassador rules around payments, tasks and 

In [99]:
# Assuming 'data' is a list of dictionaries with each dictionary representing an item
for item in data:
    # Accessing a nested key, for example, if the key is nested under 'details' key
    nested_key_value = item['meetingInfo']['host']  # Accessing the nested key
    # Now you can pass 'nested_key_value' to your query function
    # For example, you can print it
    print("Querying nested key:", nested_key_value)
    # Or perform some other operation with 'nested_key_value'


Querying nested key: Tevo
Querying nested key: Tevo
Querying nested key: LadyTempestt
Querying nested key: Headelf
Querying nested key: Tevo
Querying nested key: Headelf
Querying nested key: Rojo
Querying nested key: Rojo
Querying nested key: Slate
Querying nested key: Tevo
Querying nested key: Tevo
Querying nested key: hogantuso
Querying nested key: Stephen
Querying nested key: Peter
Querying nested key: Headelf
Querying nested key: Peter, Slate
Querying nested key: hogantuso
Querying nested key: Judith
Querying nested key: Stephen
Querying nested key: Stephen
Querying nested key: Slate
Querying nested key: Tevo
Querying nested key: Rojo, Guillermo
Querying nested key: Curtis, Guillermo
Querying nested key: Rojo, WaKa
Querying nested key: Rojo, Guillermo
Querying nested key: Headelf
Querying nested key: Judith
Querying nested key: hogantuso
Querying nested key: Peter
Querying nested key: Peter
Querying nested key: Peter
Querying nested key: Peter
Querying nested key: Peter
Querying ne